In [14]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [15]:
try:
    stop_words = set(stopwords.words("english"))
except LookupError:
    print("NLTK 'stopwords' ve 'wordnet' paketleri indiriliyor...")
    nltk.download("stopwords", quiet=True)
    nltk.download("wordnet", quiet=True)
    nltk.download('omw-1.4', quiet=True)
    stop_words = set(stopwords.words("english"))

In [16]:
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

def flatten_jsonl_data(file_path):
    """
    Konuşma bazlı JSONL dosyasını okur, boş satırları atlar ve 
    mesaj bazlı bir DataFrame'e dönüştürür.
    """
    print(f"{os.path.basename(file_path)} dosyası işleniyor...")
    
    all_messages = []
    
    # Dosyayı satır satır aç ve oku
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Satırın başındaki/sonundaki boşlukları temizle
            line = line.strip()
            
            # Eğer satır boşsa (dosya sonundaki boş satır gibi), bu satırı atla
            if not line:
                continue
            
            # Satırı bir JSON objesi olarak yükle
            row = json.loads(line)
            
            # Şimdi bu 'row' objesi (konuşma) üzerinden döngüye gir
            # (Burada sendeki 'iterrows' ve 'messages' hatalarını da düzelttim)
            for i in range(len(row["messages"])):
                message_data = {
                    'game_id': row['game_id'],
                    'speaker': row['speakers'][i],
                    'receiver': row['receivers'][i],
                    'message_text': row['messages'][i],
                    'sender_intention': row['sender_labels'][i],
                    'game_score': row['game_score'][i],
                    'game_score_delta': row['game_score_delta'][i],
                    'year': row['years'][i],
                    'season': row['seasons'][i],
                    'original_fold': row.get('acl2020_fold', os.path.basename(file_path).split('.')[0])
                }
                all_messages.append(message_data)
                
    return pd.DataFrame(all_messages)
            

In [22]:
data_folder_path = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "2020_acl_diplomacy", "data")
train_path = os.path.join(data_folder_path, "train.jsonl")
val_path = os.path.join(data_folder_path, "validation.jsonl")
test_path = os.path.join(data_folder_path, "test.jsonl")

df_train = flatten_jsonl_data(train_path)
df_val = flatten_jsonl_data(val_path)
df_test = flatten_jsonl_data(test_path)

df = pd.concat([df_train, df_val, df_test], ignore_index=True)
print("\nTüm veri setleri EDA için birleştirildi.")
print(f"Toplam mesaj (satır) sayısı: {len(df)}")

print("Veri temizleme ve özellik mühendisliği başlıyor...")
df['target'] = df['sender_intention'].apply(lambda x: 1 if x == False else 0)

df['cleaned_text'] = df['message_text'].apply(clean_text)

numeric_cols = ['game_score', 'game_score_delta', 'year']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.fillna(0, inplace=True)

df['message_length'] = df['message_text'].apply(len)
print("Temizlik tamamlandı.")

OUTPUT_DIR = os.path.join(data_folder_path, "processed")
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_FILE_PATH = os.path.join(OUTPUT_DIR, "diplomacy_processed.parquet")
print("şuraya kaydediliyor: ", OUTPUT_FILE_PATH)
df.to_parquet(OUTPUT_FILE_PATH, index=False)

print(df.info())

train.jsonl dosyası işleniyor...
validation.jsonl dosyası işleniyor...
test.jsonl dosyası işleniyor...

Tüm veri setleri EDA için birleştirildi.
Toplam mesaj (satır) sayısı: 17289
Veri temizleme ve özellik mühendisliği başlıyor...
Temizlik tamamlandı.
şuraya kaydediliyor:  c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\processed\diplomacy_processed.parquet
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17289 entries, 0 to 17288
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   game_id           17289 non-null  int64 
 1   speaker           17289 non-null  object
 2   receiver          17289 non-null  object
 3   message_text      17289 non-null  object
 4   sender_intention  17289 non-null  bool  
 5   game_score        17289 non-null  int64 
 6   game_score_delta  17289 non-null  int64 
 7   year              17289 non-null  int64 
 8   season            17289 non-null 

In [18]:
base_path = os.path.dirname(os.getcwd())
data_path = os.path.join(base_path, "data", "raw", "2020_acl_diplomacy", "data", "processed", "diplomacy_eda_processed.parquet")
reports_dir = os.path.join(base_path, "reports", "figures")
os.makedirs(reports_dir, exist_ok=True)

In [19]:
try:
    data = pd.read_parquet(data_path)
    print(f"'{data_path}' dosyasından {len(data)} satır veri başarıyla yüklendi.")
    print(f"Grafikler '{reports_dir}' klasörüne kaydedilecek.")
    # Grafik stilini ayarla
    sns.set_theme(style="whitegrid")
except FileNotFoundError:
    print(f"HATA: '{data_path}' dosyası bulunamadı!")
    print("Lütfen bir önceki ön işleme (preprocessing) kodunu çalıştırdığından emin ol.")

'c:\work environment\Projects\amazon-spam-review\data\raw\2020_acl_diplomacy\data\processed\diplomacy_eda_processed.parquet' dosyasından 17289 satır veri başarıyla yüklendi.
Grafikler 'c:\work environment\Projects\amazon-spam-review\reports\figures' klasörüne kaydedilecek.
